<a href="https://colab.research.google.com/github/kimdong799/langchain-tutorial/blob/main/LangChain_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CH01 LangChain 시작하기

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# AI 답변 확인용
from IPython.display import Markdown

import os
os.environ["GOOGLE_API_KEY"] = "MY API KEY"